<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Test-Case:-A-Motivating-Example:-Linear-Regression" data-toc-modified-id="Test-Case:-A-Motivating-Example:-Linear-Regression-0.1">Test Case: A Motivating Example: Linear Regression</a></span><ul class="toc-item"><li><span><a href="#Running-with-Theano-backend-(restart-kernel-for-a-fresh-import)" data-toc-modified-id="Running-with-Theano-backend-(restart-kernel-for-a-fresh-import)-0.1.1">Running with Theano backend (restart kernel for a fresh import)</a></span></li><li><span><a href="#Running-with-TensorFlow-backend-(restart-kernel-for-a-fresh-import)" data-toc-modified-id="Running-with-TensorFlow-backend-(restart-kernel-for-a-fresh-import)-0.1.2">Running with TensorFlow backend (restart kernel for a fresh import)</a></span></li></ul></li></ul></li></ul></div>

## Test Case: A Motivating Example: Linear Regression

### Running with Theano backend (restart kernel for a fresh import)

Here I import the local PyMC3 package that I changed. I set the backend to be Theano. 

In [1]:
import os
os.chdir('../../../')
print(os.getcwd())
import pymc3 as pm
import numpy as np

/Users/chris/Desktop/pymc3


Using Theano backend.


In [2]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [3]:
basic_model2 = pm.Model()
with basic_model2:
    alpha = pm.Normal(name='alpha',mu=0.0, sd=10.0) 
    beta = pm.Normal(name='beta', mu=0.0, sd=10.0, shape=2)

    mu = alpha + beta[0]*X1 + beta[1]*X2

    Y_obs = pm.Normal(name='Y_obs',mu=mu, sd=1.0, observed=Y)

    #step = pm.Metropolis()
    step = pm.NUTS()
    trace = pm.sample(5000,step=step,cores=1,chains=1) # switched off parallel for simplifications. 

print(np.mean(trace['alpha']))
print(np.mean(trace['beta'],axis=0))

> /Users/chris/Desktop/pymc3/pymc3/model.py(455)__init__()
-> grad = tt.grad(self._cost_joined, self._vars_joined)
(Pdb) n
> /Users/chris/Desktop/pymc3/pymc3/model.py(459)__init__()
-> grad.name = '__grad'
(Pdb) n
> /Users/chris/Desktop/pymc3/pymc3/model.py(461)__init__()
-> inputs = [self._vars_joined]
(Pdb) n
> /Users/chris/Desktop/pymc3/pymc3/model.py(472)__init__()
-> self._theano_function = S.function(inputs, [self._cost_joined, grad], givens=givens, **kwargs)
(Pdb) n
--Return--
> /Users/chris/Desktop/pymc3/pymc3/model.py(472)__init__()->None
-> self._theano_function = S.function(inputs, [self._cost_joined, grad], givens=givens, **kwargs)
(Pdb) n
--Return--
> /Users/chris/Desktop/pymc3/pymc3/model.py(743)logp_dlogp_function()-><pymc3.model.... 0x1c1d47f3c8>
-> return ValueGradFunction(self.logpt, grad_vars, extra_vars, **kwargs)
(Pdb) n
--Return--
> /Users/chris/Desktop/pymc3/pymc3/step_methods/arraystep.py(231)__init__()->None
-> vars, dtype=dtype, **theano_kwargs)
(Pdb) n
> /Use

BdbQuit: 

### Running with TensorFlow backend (restart kernel for a fresh import)

After restarting the kernel, I import the local PyMC3 package again, but now I set the backend to be TensorFlow.

In [1]:
import os
os.chdir('../../../')
print(os.getcwd())
import numpy as np
os.environ["PYMC_SYMB_BACKEND"] = "tensorflow" 
import pymc3 as pm

/Users/chris/Desktop/pymc3


Using TensorFlow backend.


In [2]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [3]:
basic_model2 = pm.Model()
with basic_model2:
    
    # requires initial value now. 
    # also has to be float at the moment. 
    alpha = pm.Normal(name='alpha',initial_value=0.0, mu=0.0, sd=10.0) 
    beta = pm.Normal(name='beta',initial_value=np.array([1.0,1.0],dtype='float32'), mu=0.0, sd=10.0, shape=2)

    mu = alpha + beta[0]*X1 + beta[1]*X2

    Y_obs = pm.Normal(name='Y_obs',initial_value=np.ones_like(Y), mu=mu, sd=1.0, observed=Y)

    step = pm.Metropolis()
    trace = pm.sample(5000,step=step,cores=1,chains=1)

print(np.mean(trace['alpha:0']))
print(np.mean(trace['beta:0'],axis=0))

Sequential sampling (1 chains in 1 job)
CompoundStep
>Metropolis: [<tf.Variable 'alpha:0' shape=() dtype=float32_ref>]
>Metropolis: [<tf.Variable 'beta:0' shape=(2,) dtype=float32_ref>]
100%|██████████| 5500/5500 [00:12<00:00, 454.68it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


0.905485
[ 0.94781202  2.62093997]
